In [ ]:
# Importando as bibliotecas necessárias

import pandas as pd # Para Manipulação dos Dados

# Para se conectar no SQL SERVER
import pyodbc as sql

In [ ]:
# Conectando no Servidor - Servidor Local (SQL SERVER/SSMS)

dados_conection = (
    "Driver={SQL SERVER};"
    "Server=Servidor;"
)

conexao = sql.connect(dados_conection)
print("Funcionou")

In [3]:
# cursor para conseguir codificar em SQL
cursor = conexao.cursor()

In [ ]:
# Criando o Banco de Dados

nome_DB = 'FinanceX Solutions' # Nome fictício da empresa

criacao_DB = f"CREATE DATABASE [{nome_DB}]"

cursor.execute(criacao_DB)
print("Banco de Dados Criado!")

Banco de Dados Criado!


In [6]:
# Selecionando o DB
cursor.execute(f"USE [{nome_DB}]")
print(f"Usando o banco de dados {nome_DB}")

Usando o banco de dados FinanceX Solutions


In [ ]:
# Criando a tabela Clientes

tabela1 = """
CREATE TABLE Clientes (
    id_cliente INT PRIMARY KEY,
    nome NVARCHAR(100),
    email NVARCHAR(100),
    telefone NVARCHAR(20),
    empresa NVARCHAR(100)
);
"""

cursor.execute(tabela1)
conexao.commit()
print("Tabela criada!")

Tabela criada!


In [ ]:
# Criando a tabela Faturas

tabela2 = """
CREATE TABLE Faturas (
    id_fatura INT PRIMARY KEY,
    id_cliente INT,
    data_emissao DATE,
    data_vencimento DATE,
    valor_total DECIMAL(10, 2),
    status NVARCHAR(20),
    FOREIGN KEY (id_cliente) REFERENCES Clientes(id_cliente)
);

"""

cursor.execute(tabela2)
conexao.commit()
print("Tabela criada")

Tabela criada


In [32]:
# Criando a tabela Pagamentos

tabela3 = """
CREATE TABLE Pagamentos (
    id_pagamento INT PRIMARY KEY,
    id_fatura INT,
    data_pagamento DATE,
    valor_pago DECIMAL(10, 2),
    metodo_pagamento NVARCHAR(50),
    FOREIGN KEY (id_fatura) REFERENCES Faturas(id_fatura)
);

"""

cursor.execute(tabela3)
conexao.commit()
print("Tabela criada")

Tabela criada


In [33]:
# Criando a tabela Despesas

tabela4 = """
CREATE TABLE Despesas (
    id_despesa INT PRIMARY KEY,
    categoria NVARCHAR(50),
    descricao NVARCHAR(255),
    valor DECIMAL(10, 2),
    data_despesa DATE
);

"""

cursor.execute(tabela4)
conexao.commit()
print("Tabela criada")

Tabela criada


In [34]:
# Criando a tabela Receitas

tabela5 = """
CREATE TABLE Receitas (
    id_receita INT PRIMARY KEY,
    id_cliente INT,
    descricao NVARCHAR(255),
    valor DECIMAL(10, 2),
    data_recebimento DATE,
    FOREIGN KEY (id_cliente) REFERENCES Clientes(id_cliente)
);

"""

cursor.execute(tabela5)
conexao.commit()
print("Tabela criada")

Tabela criada


In [ ]:
# Importando os Dados

clientes = pd.read_csv("caminho/clientes.csv")
despesas = pd.read_csv("caminho/despesas.csv")
faturas = pd.read_csv("caminho/faturas.csv")
pagamentos = pd.read_csv("caminho/pagamentos.csv")
receitas = pd.read_csv("caminho/receitas.csv")


In [ ]:
# Inserindo Dados nas tabelas

#Clientes
dados1 = [(row['id_cliente'], row['nome'], row['email'], row['telefone'], row['empresa']) for i, row in clientes.iterrows()]

insert1 = """
INSERT INTO dbo.Clientes (id_cliente, nome, email, telefone, empresa)
VALUES (?, ?, ?, ?, ?)
"""

# .executemany --> Executa uma mesmo consulta várias vezes com valores diferentes em cada execução
cursor.executemany(insert1, dados1)
cursor.commit()

In [ ]:
# ERROR🔺

# Os valores de "telefone" tem mais caracteres do que a quantidade definida na criação da tabela "Clientes".
# Utizando o ALTER TABLE e ALTER COLUMN para correção

modific_telefone = """
ALTER TABLE dbo.Clientes
ALTER COLUMN telefone NVARCHAR(30);
"""
cursor.execute(modific_telefone)
cursor.commit()
print("Alterado")

# Em seguida excluimos os valores que já foram importados para a tabela.

deletando = """
DELETE FROM dbo.Clientes
"""

cursor.execute(deletando)
cursor.commit()
print("Valores deletado")

Alterado
Valores deletado


In [ ]:
# Continuação - Inserção de Dados

dados2 = [(row['id_fatura'], row['id_cliente'], row['data_emissao'], row['data_vencimento'], row['valor_total'], row['status']) for i, row in faturas.iterrows()]

insert2 = """
INSERT INTO dbo.Faturas (id_fatura, id_cliente, data_emissao, data_vencimento, valor_total, status)
VALUES(?, ?, ?, ?, ?, ?)
"""

cursor.executemany(insert2, dados2)
cursor.commit()


dados3 = [(row['id_receita'], row['id_cliente'], row['descricao'], row['valor'], row['data_recebimento']) for i, row in receitas.iterrows()]

insert3 = """
INSERT INTO dbo.Receitas (id_receita, id_cliente, descricao, valor, data_recebimento)
VALUES(?, ?, ?, ?, ?)
"""

cursor.executemany(insert3, dados3)
cursor.commit()


dados4 = [(row['id_despesa'], row['categoria'], row['descricao'], row['valor'], row['data_despesa']) for i, row in despesas.iterrows()]

insert4 = """
INSERT INTO dbo.Despesas (id_despesa, categoria, descricao, valor, data_despesa)
VALUES(?, ?, ?, ?, ?)
"""

cursor.executemany(insert4, dados4)
cursor.commit()


dados5 = [(row['id_pagamento'], row['id_fatura'], row['data_pagamento'], row['valor_pago'], row['metodo_pagamento']) for i, row in pagamentos.iterrows()]

insert5 = """
INSERT INTO dbo.Pagamentos (id_pagamento,id_fatura,data_pagamento,valor_pago,metodo_pagamento)
VALUES(?, ?, ?, ?, ?)
"""

cursor.executemany(insert5, dados5)
cursor.commit()

In [ ]:
# Encerrando a conexão com o Servidor
cursor.close()
conexao.close()
print("Conexão encerrada")

Conexão encerrada
